In [1]:
3+4

7

In [1]:

%pwd

'c:\\Users\\Star\\Desktop\\github\\Github\\GenAI\\GenerativeAI_Projects\\End_To_End_MedicalChatBot\\research'

In [2]:
import os
os.chdir('../')

%pwd

'c:\\Users\\Star\\Desktop\\github\\Github\\GenAI\\GenerativeAI_Projects\\End_To_End_MedicalChatBot'

In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [4]:
%pwd

import os
os.chdir('../')

%pwd

'c:\\Users\\Star\\Desktop\\github\\Github\\GenAI\\GenerativeAI_Projects\\End_To_End_MedicalChatBot'

In [5]:
extract_data = load_pdf(data='Data/')

In [31]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [34]:
text_chunks = text_split(extract_data)
print("Length of text chunks ",len(text_chunks))

Length of text chunks  7298


In [43]:
text_chunks[3]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\medical_book.pdf', 'total_pages': 637, 'page': 3, 'page_label': '4'}, page_content='Content\nRobyn V . Young,Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Content\nKelly A. Quin, Editor, Imaging and Multimedia Content\nLeitha Etheridge-Sims, Mary K. Grimes, Dave Oblender,\nImage Catalogers\nPamela A. Reed, Imaging Coordinator\nRandy Bassett, Imaging Supervisor\nRobert Duncan, Senior Imaging Specialist')

In [35]:
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings


In [36]:
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [37]:
# List all available models
models = genai.list_models()

# Filter only embedding models
embedding_models = [model for model in models if "embedding" in model.name.lower()]

for model in embedding_models:
    print(model.name)

models/embedding-gecko-001
models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [38]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [39]:
# to check the dimension of the embedding model 
# later that has to be sepecified in pinecone index creation

ansvec = embeddings.embed_query("how are you?") # pass any sentence or word to check the dimension
print("Length", len(ansvec))
model_dimension = len(ansvec)
#768 will be the dimension we get for this embedding model

Length 768


In [40]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [41]:
#to create a index in pinecode via code

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=model_dimension, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': 'efd3e119345808983ed3ba920ee4c46e', 'date': 'Sat, 26 Apr 2025 13:21:21 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [48]:
print(pc.Index)

<bound method PineconeGRPC.Index of <pinecone.grpc.pinecone.PineconeGRPC object at 0x000001EAAC429F60>>


In [51]:
from langchain_pinecone import PineconeVectorStore

def batch(iterable, batch_size=50):
    for i in range(0, len(iterable), batch_size):
        yield iterable[i:i + batch_size]

for chunk_batch in batch(text_chunks, batch_size=50):
    PineconeVectorStore.from_documents(
        documents=chunk_batch,
        index_name=index_name,
        embedding=embeddings
    )

print("✅ Uploaded safely in manual batches!")


✅ Uploaded safely in manual batches!


In [52]:
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name,embedding=embeddings)

In [55]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [56]:
retriever.invoke("hii")

[Document(id='091345d9-437d-4b88-a8c6-627bc9ad8c10', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 15.0, 'page_label': '16', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\medical_book.pdf', 'total_pages': 637.0}, page_content='they pass. These vibrations are mild, virtually unnotice-\nable at the frequencies and intensities used for imaging.\nProperly focused however, high-intensity ultrasound can\nbe used to heat and physically agitate targeted tissues.\nHigh-intensity ultrasound is used routinely to treat\nsoft tissue injuries, such as strains, tears and associated\nscarring. The heating and agitation are believed to pro-'),
 Document(id='fceb0463-43b9-4fc3-bde7-1879a9bc2813', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 322.0, 'page_label': '323', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\medical_book.pdf

In [57]:
llm = GoogleGenerativeAI(model="models/gemini-2.0-flash")

In [58]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [59]:
system_prompt = """You are an assistant for question answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.


{context}"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
         ("human", "{input}")]
)

In [60]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [61]:
resp = rag_chain.invoke({"input":"What are the products it has"})
resp['answer']

'Based on the provided text, the products mentioned are Benoxyl, Clear By Design, Neutrogena Acne, PanOxyl, and some formulations of Clean & Clear, and Clearasil. These products contain benzoyl peroxide and are used for treating acne. The Gale Encyclopedia of Medicine 2 contains information on disorders, tests, procedures, and treatments.\n'

In [62]:
resp = rag_chain.invoke({"input":"What is acne"})
resp['answer']

'Acne vulgaris is a common skin disease affecting nearly 17 million people in the United States. It arises at any age, usually beginning at puberty and worsening during adolescence. The most common sites of acne are the face, chest, shoulders, and back.\n'

In [63]:
resp = rag_chain.invoke({"input":"hi good evening"})
resp['answer']

'How can I help you this evening?\n'

In [64]:
resp = rag_chain.invoke({"input":"Who are u?"})
resp['answer']

"I am an assistant for question answering tasks. I use retrieved context to answer questions. If I don't know the answer, I will say so.\n"

In [68]:
resp = rag_chain.invoke({"input":"What are disease related to heart?"})
resp['answer']

'Infection by a bacterium can damage the heart and its valves. Smoking is also detrimental to heart health. Additionally, high blood pressure can affect the heart.\n'

In [66]:
resp = rag_chain.invoke({"input":"how to cure that?"})
resp['answer']

'Please provide me with the context about the condition you are asking about. I need more information to provide a relevant answer about treatment options.\n'

In [69]:
resp = rag_chain.invoke({"input":"how to cure disease related to heart?"})
resp['answer']

'To address heart-related diseases, both conventional and alternative medicine emphasize the importance of lifestyle adjustments. Key steps include regular exercise and dietary modifications. These modifications involve reducing sugar and saturated fats while increasing dietary fiber intake.\n'